In [392]:
import pandas as pd
import plotly.express as px
import numpy as np

In [393]:
T_RANGE = 6
Y_RANGE = [-100, 100]
HEIGHT = 600

In [394]:

def trimmed_rms(series, lower_percentile=5, upper_percentile=95):
    # Remove NaNs just in case
    clean_series = series.dropna()

    # Compute lower and upper bounds
    lower = np.percentile(clean_series, lower_percentile)
    upper = np.percentile(clean_series, upper_percentile)

    # Filter out outliers
    trimmed = clean_series[(clean_series >= lower) & (clean_series <= upper)]

    # Compute RMS on trimmed data
    return np.sqrt((trimmed ** 2).mean())

## Solarflare

### Gradmaster

In [395]:
column_names = ['date', 'time', 'system', 'offset_title', 'offset', 'adj_title', 'adj', 'sync_title', 'sync']
df = pd.read_csv('sfptp-gm.log', delim_whitespace=True, names=column_names, header=None)

df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df['offset'] = df['offset'].str.rstrip(',').astype(float)
now = df['datetime'].max()
df = df[df['datetime'] >= now - pd.Timedelta(minutes=T_RANGE)]

rms = np.sqrt((df['offset'] ** 2).mean())

#print(df.head())

In [396]:
fig = px.line(df, x='datetime', y='offset', title='Solarflare PHC Offset - GM',
                 labels={'datetime': 'Timestamp', 'offset': 'Offset (ns)'}, height=HEIGHT)
fig.update_layout(yaxis=dict(range=Y_RANGE))

fig.add_hline(y=rms, line_dash='dash', line_color='orange', annotation_text=f'+RMS {round(rms,2)}', annotation_position='top left')
fig.add_hline(y=-rms, line_dash='dash', line_color='orange', annotation_text=f'-RMS {round(-rms,2)}', annotation_position='bottom left')

fig.show()

### Ordinary Clock 

#### PHC

In [397]:
column_names = ['date', 'time', 'system', 'offset_title', 'offset', 'adj_title', 'adj', 'sync_title', 'sync']
df = pd.read_csv('sfptp-slave-phc.log', delim_whitespace=True, names=column_names, header=None)

df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df['offset'] = df['offset'].str.rstrip(',').astype(float)
now = df['datetime'].max()
df = df[df['datetime'] >= now - pd.Timedelta(minutes=T_RANGE)]

rms = np.sqrt((df['offset'] ** 2).mean())

#print(df.head())

In [398]:
fig = px.line(df, x='datetime', y='offset', title='Solarflare PHC Offset - OC',
                 labels={'datetime': 'Timestamp', 'offset': 'Offset (ns)'}, height=HEIGHT)
fig.update_layout(yaxis=dict(range=Y_RANGE))

fig.add_hline(y=rms, line_dash='dash', line_color='orange', annotation_text=f'+RMS {round(rms,2)}', annotation_position='top left')
fig.add_hline(y=-rms, line_dash='dash', line_color='orange', annotation_text=f'-RMS {round(-rms,2)}', annotation_position='bottom left')


fig.show()

#### PTP

In [399]:
column_names = ['date', 'time', 'system', 'offset_title', 'offset', 'adj_title', 'adj', 'sync_title', 'sync', 'delay_title', 'delay', 'parent_title', 'parent', 'gm_title', 'gm']
df = pd.read_csv('sfptp-slave-ptp.log', delim_whitespace=True, names=column_names, header=None)

df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df['offset'] = df['offset'].str.rstrip(',').astype(float)
now = df['datetime'].max()
df = df[df['datetime'] >= now - pd.Timedelta(minutes=T_RANGE)]

rms = np.sqrt((df['offset'] ** 2).mean())

#print(df.head())

In [400]:
fig = px.line(df, x='datetime', y='offset', title='Solarflare PTP offset - OC',
                 labels={'datetime': 'Timestamp', 'offset': 'Offset (ns)'}, height=HEIGHT)
fig.update_layout(yaxis=dict(range=Y_RANGE))

fig.add_hline(y=rms, line_dash='dash', line_color='orange', annotation_text=f'+RMS {round(rms,2)}', annotation_position='top left')
fig.add_hline(y=-rms, line_dash='dash', line_color='orange', annotation_text=f'-RMS {round(-rms,2)}', annotation_position='bottom left')

fig.show()

## Linuxptp

### Grandmaster

#### PHC2SYS

In [401]:
column_names = ['Month' ,'day', 'time', 'host', 'process', 'system', 'interface', 'sys_title','offset_title', 'offset', 's', 'adj_title', 'adj', 'delay_title', 'delay']
gm_phc_df = pd.read_csv('phc2sys-gm.log', delim_whitespace=True, names=column_names, header=None)

# Remove rows where 'column_name' contains "test"
gm_phc_df = gm_phc_df[~gm_phc_df['system'].str.contains('phc2sys', case=False, na=False)]


gm_phc_df['datetime'] = pd.to_datetime(
    '2025 ' + gm_phc_df['Month'] + ' ' + gm_phc_df['day'].astype(str) + ' ' + gm_phc_df['time'],
    format='%Y %b %d %H:%M:%S'
)
#df['offset'] = df['offset'].str.rstrip(',').astype(float)
now = gm_phc_df['datetime'].max()
gm_phc_df = gm_phc_df[gm_phc_df['datetime'] >= now - pd.Timedelta(minutes=T_RANGE)]

rms = np.sqrt((gm_phc_df['offset'] ** 2).mean())

#print(df.tail())

In [402]:
fig = px.line(gm_phc_df, x='datetime', y='offset', title='LinuxPTP PHC offset - GM',
                 labels={'datetime': 'Timestamp', 'offset': 'Offset (ns)'}, height=HEIGHT)
fig.update_layout(yaxis=dict(range=Y_RANGE))

fig.add_hline(y=rms, line_dash='dash', line_color='orange', annotation_text=f'+RMS {round(rms,2)}', annotation_position='top left')
fig.add_hline(y=-rms, line_dash='dash', line_color='orange', annotation_text=f'-RMS {round(-rms,2)}', annotation_position='bottom left')

fig.show()

### Ordinary Clock 

In [403]:
column_names = ['Month' ,'day', 'time', 'host', 'process', 'system', 'interface', 'sys_title','offset_title', 'offset', 's', 'adj_title', 'adj', 'delay_title', 'delay']
slave_phc_df = pd.read_csv('phc2sys-slave.log', delim_whitespace=True, names=column_names, header=None)

# Remove rows where 'column_name' contains "test"
df = slave_phc_df[~slave_phc_df['system'].str.contains('phc2sys', case=False, na=False)]


slave_phc_df['datetime'] = pd.to_datetime(
    '2025 ' + slave_phc_df['Month'] + ' ' + slave_phc_df['day'].astype(str) + ' ' + slave_phc_df['time'],
    format='%Y %b %d %H:%M:%S'
)
#df['offset'] = df['offset'].str.rstrip(',').astype(float)
now = slave_phc_df['datetime'].max()
slave_phc_df = slave_phc_df[slave_phc_df['datetime'] >= now - pd.Timedelta(minutes=T_RANGE)]

#rms = np.sqrt((slave_phc_df['offset'] ** 2).mean())
rms = trimmed_rms(slave_phc_df['offset'], lower_percentile=5, upper_percentile=95)

#print(df.tail())

In [404]:
fig = px.line(slave_phc_df, x='datetime', y='offset', title='LinuxPTP PHC offset - OC',
                 labels={'datetime': 'Timestamp', 'offset': 'Offset (ns)'}, height=HEIGHT)
fig.update_layout(yaxis=dict(range=Y_RANGE))

fig.add_hline(y=rms, line_dash='dash', line_color='orange', annotation_text=f'+RMS {round(rms,2)}', annotation_position='top left')
fig.add_hline(y=-rms, line_dash='dash', line_color='orange', annotation_text=f'-RMS {round(-rms,2)}', annotation_position='bottom left')

fig.show()

#### ptp4l

In [405]:
column_names = ['Month' ,'day', 'time', 'host', 'process', 'system', 'interface','offset_title', 'offset', 's', 'adj_title', 'adj', 'path_title', 'delay_title', 'delay']
df = pd.read_csv('ptp4l-slave.log', delim_whitespace=True, names=column_names, header=None)

# Remove rows where 'column_name' contains "test"
df = df[~df['system'].str.contains('ptp4l', case=False, na=False)]
df['offset'] = pd.to_numeric(df['offset'].astype(str).str.rstrip(','), errors='coerce')
df = df.dropna(subset=['offset'])

df['datetime'] = pd.to_datetime(
    '2025 ' + df['Month'] + ' ' + df['day'].astype(str) + ' ' + df['time'],
    format='%Y %b %d %H:%M:%S'
)
#df['offset'] = df['offset'].str.rstrip(',').astype(float)
df = df[df['datetime'] >= now - pd.Timedelta(minutes=T_RANGE)]
df = df[df['datetime'] <= now]

#rms = np.sqrt((df['offset'] ** 2).mean())
rms = trimmed_rms(df['offset'], lower_percentile=5, upper_percentile=95)

#print(df.tail())

In [406]:
fig = px.line(df, x='datetime', y='offset', title='LinuxPTP PTP offset - OC',
                 labels={'datetime': 'Timestamp', 'offset': 'Offset (ns)'}, height=HEIGHT)
fig.update_layout(yaxis=dict(range=Y_RANGE))

fig.add_hline(y=rms, line_dash='dash', line_color='orange', annotation_text=f'+RMS {round(rms,2)}', annotation_position='top left')
fig.add_hline(y=-rms, line_dash='dash', line_color='orange', annotation_text=f'-RMS {round(-rms,2)}', annotation_position='bottom left')

fig.show()

In [407]:
gm_phc_df['source'] = 'GM'
slave_phc_df['source'] = 'SLAVE'

df_combined = pd.concat([gm_phc_df, slave_phc_df])


In [408]:
#fig = px.line(df_combined, x='datetime', y='offset', title='Offset over Time', color='source',
#                 labels={'datetime': 'Timestamp', 'offset': 'Offset (ns)'}, height=HEIGHT)
#fig.update_layout(yaxis=dict(range=Y_RANGE))
#
#fig.show()